# Chunking data từ file pdf vô ChromaDB
## 1. Import thư viện cần thiết

In [9]:
# pip install langchain langchain-community chromadb jupyter sentence-transformers unstructured[all] pytesseract pillow pdf2image
import warnings
from langchain_community.document_loaders import UnstructuredPDFLoader # Load PDF phức tạp, hỗ trợ OCR cho PDF scan
from langchain_text_splitters import RecursiveCharacterTextSplitter # Chia văn bản thành các chunks nhỏ
from langchain_community.embeddings import HuggingFaceEmbeddings # Sử dụng mô hình embedding từ HuggingFace
from langchain_community.vectorstores import Chroma # Vector database để lưu trữ và tìm kiếm embeddings
from langchain_text_splitters import MarkdownTextSplitter
import re # Thêm thư viện re để làm sạch
import os # Thư viện hệ thống

# Bỏ qua các cảnh báo không quan trọng
warnings.filterwarnings('ignore')

print("✅ Đã import thành công tất cả thư viện LangChain!")

✅ Đã import thành công tất cả thư viện LangChain!


## 2. Cấu hình đường dẫn Tesseract OCR và Poppler

In [10]:

# Tải Tesseract tại: github.com/UB-Mannheim/tesseract/wiki
# Tải Poppler tại: github.com/oschwartz10612/poppler-windows/releases

# 1. Định nghĩa đường dẫn ĐẾN folder cài đặt Tesseract và Poppler, không phải file .exe
#    Đảm bảo đường dẫn này CHÍNH XÁC với nơi bạn cài đặt
tesseract_dir = r"C:\Program Files\Tesseract-OCR"
poppler_dir = r"C:\poppler-25.07.0\Library\bin" # Thư mục 'bin' là chính xác

# 2. Lấy biến PATH hiện tại của hệ thống
current_path = os.environ.get("PATH", "")

# 3. Kiểm tra đường dẫn của Tesseract và Poppler
if os.path.exists(tesseract_dir):
    print(f"✅ Đã tìm thấy Tesseract tại: {tesseract_dir}")
else:
    print(f"❌ CẢNH BÁO: Không tìm thấy Tesseract tại: {tesseract_dir}")
    print("Vui lòng kiểm tra lại đường dẫn cài đặt.")

if os.path.exists(poppler_dir):
    print(f"✅ Đã tìm thấy Poppler tại: {poppler_dir}")
else:
    print(f"❌ CẢNH BÁO: Không tìm thấy Poppler tại: {poppler_dir}")
    print("Vui lòng kiểm tra lại đường dẫn cài đặt.")

# 4. Thêm Tesseract và Poppler vào PATH (lấy PATH đã cập nhật ở trên)
#    Quan trọng: Cập nhật lại current_path để nó chứa các thay đổi mới nhất

if tesseract_dir not in current_path:
    # os.pathsep là dấu ; trên Windows, dùng để nối các đường dẫn
    os.environ["PATH"] = current_path + os.pathsep + tesseract_dir
    print(f"✅ Đã thêm Tesseract vào PATH: {tesseract_dir}")
else:
    print("ℹ️ Tesseract đã có trong PATH.")
    

current_path_with_tesseract = os.environ.get("PATH", "")
if poppler_dir not in current_path_with_tesseract:
    os.environ["PATH"] = current_path_with_tesseract + os.pathsep + poppler_dir
    print(f"✅ Đã thêm Poppler vào PATH: {poppler_dir}")
else:
    print("ℹ️ Poppler đã có trong PATH.")

# 5. Kiểm tra (Bạn có thể bỏ qua phần in này nếu muốn)
print("\n--- Biến PATH sau khi cập nhật (100 ký tự cuối) ---")
print("..." + os.environ.get("PATH", "")[-100:])

✅ Đã tìm thấy Tesseract tại: C:\Program Files\Tesseract-OCR
✅ Đã tìm thấy Poppler tại: C:\poppler-25.07.0\Library\bin
ℹ️ Tesseract đã có trong PATH.
ℹ️ Poppler đã có trong PATH.

--- Biến PATH sau khi cập nhật (100 ký tự cuối) ---
...esources\app\bin;C:\Users\Carlo\go\bin;C:\Program Files\Tesseract-OCR;C:\poppler-25.07.0\Library\bin


In [11]:
# Truong
from langchain_docling import DoclingLoader

## 3. Load tài liệu PDF với UnstructuredPDFLoader

In [12]:
# 1. Trỏ đến thư mục chứa data
data_dir = "./data" 
documents = [] # List rỗng để chứa TẤT CẢ docs từ các file

print(f"Bắt đầu quét thư mục: {os.path.abspath(data_dir)}")

if not os.path.exists(data_dir):
    raise FileNotFoundError(f"❌ Không tìm thấy thư mục data tại: {data_dir}")

# 2. Lặp qua tất cả các file trong thư mục
for filename in os.listdir(data_dir):
    if filename.endswith(".pdf"): # Chỉ xử lý file PDF
        file_path = os.path.join(data_dir, filename)
        
        print(f"📄 Đang tải file: {filename}...")
        
        # 3. Khởi tạo loader cho từng file
        loader = UnstructuredPDFLoader(
            file_path,
            # mode="elements",  # Phân tích PDF thành các elements riêng lẻ (đoạn văn, bảng, tiêu đề...) thay vì xử lý nguyên trang => không phù hợp với proj
            mode="single", # Xử lý nguyên trang, phù hợp với tài liệu đơn giản, sau đó chunking bằng text splitter sau
            strategy="hi_res", # Chiến lược độ phân giải cao, tốt cho tables và layouts phức tạp, không thể dùng với "fast"
            languages=["vie"], # Ngôn ngữ tiếng Việt để OCR chính xác hơn (default là "eng")
            pdf_infer_table_structure=True  # Yêu cầu unstructured cố gắng phân tích bảng
        )
        
        try:
            # 4. Tải file và thêm vào list chung
            # Lưu ý: Đảm bảo đã cài đặt các thư viện cần thiết và poppler trên hệ thống
            # pip install pdfminer.six pi-heif unstructured-inference unstructured-pytesseract
            loaded_docs_from_file = loader.load() # trả về một list (dù mode="single" nó vẫn là list 1 phần tử)
            documents.extend(loaded_docs_from_file) # Dùng .extend() để thêm tất cả phần tử từ list con vào list lớn
            print(f"✅ Đã tải xong: {filename}")
        except Exception as e:
            # Bỏ qua file lỗi và đi tiếp
            print(f"❌ LỖI khi tải file {filename}: {e}. Bỏ qua file này.")

# 5. Kiểm tra kết quả
print(f"🎉🎉🎉 HOÀN TẤT: Đã tải thành công {len(documents)} file PDF.")

Bắt đầu quét thư mục: d:\LearnUniversity\Semester11\TieuLuanChuyenNganh\LLM_University_Policy_Chatbot\data
📄 Đang tải file: 1238 qd ban hanh quy dinh cac hoc phan ngoai ngu trong chuong trinh dao tao trinh do DH tu khoa 2023.pdf...


2025-11-14 08:39:25,709 - INFO - PDF text extraction failed, skip text extraction...
2025-11-14 08:39:26,985 - INFO - Reading PDF for file: ./data\1238 qd ban hanh quy dinh cac hoc phan ngoai ngu trong chuong trinh dao tao trinh do DH tu khoa 2023.pdf ...
2025-11-14 08:39:38,152 - INFO - PDF text extraction failed, skip text extraction...
2025-11-14 08:39:38,152 - INFO - Reading PDF for file: ./data\16.9.2025_TB triển khai KS SV về HĐGD của GV_HKI 25-26 final.signed.pdf ...


✅ Đã tải xong: 1238 qd ban hanh quy dinh cac hoc phan ngoai ngu trong chuong trinh dao tao trinh do DH tu khoa 2023.pdf
📄 Đang tải file: 16.9.2025_TB triển khai KS SV về HĐGD của GV_HKI 25-26 final.signed.pdf...


2025-11-14 08:39:40,161 - INFO - PDF text extraction failed, skip text extraction...
2025-11-14 08:39:40,161 - INFO - Reading PDF for file: ./data\2692_Thông báo vv kế hoạch đăng ký và chỉnh sửa học phần qua mạng học kỳ I, năm học 2025-2026 (dành cho sinh viên chính quy hệ đại trà và CLC TV tại trường).pdf ...


✅ Đã tải xong: 16.9.2025_TB triển khai KS SV về HĐGD của GV_HKI 25-26 final.signed.pdf
📄 Đang tải file: 2692_Thông báo vv kế hoạch đăng ký và chỉnh sửa học phần qua mạng học kỳ I, năm học 2025-2026 (dành cho sinh viên chính quy hệ đại trà và CLC TV tại trường).pdf...


...

✅ Đã tải xong: Thông báo nhận đơn xin thay thế môn học.pdf
📄 Đang tải file: Thông báo nhận đơn xin điểm I - đợt 1 HK1.25-26.pdf...


2025-11-14 08:41:35,785 - INFO - PDF text extraction failed, skip text extraction...
2025-11-14 08:41:35,785 - INFO - Reading PDF for file: ./data\Thông báo nộp lệ phí làm bằng TN ĐHCQ đợt 5.2025.pdf ...


✅ Đã tải xong: Thông báo nhận đơn xin điểm I - đợt 1 HK1.25-26.pdf
📄 Đang tải file: Thông báo nộp lệ phí làm bằng TN ĐHCQ đợt 5.2025.pdf...


2025-11-14 08:41:39,781 - INFO - PDF text extraction failed, skip text extraction...
2025-11-14 08:41:39,782 - INFO - Reading PDF for file: ./data\Thông báo đăng ký miễn thi giáo dục thể chất, giáo dục quốc phòng đối với khóa 2025.pdf ...


✅ Đã tải xong: Thông báo nộp lệ phí làm bằng TN ĐHCQ đợt 5.2025.pdf
📄 Đang tải file: Thông báo đăng ký miễn thi giáo dục thể chất, giáo dục quốc phòng đối với khóa 2025.pdf...
✅ Đã tải xong: Thông báo đăng ký miễn thi giáo dục thể chất, giáo dục quốc phòng đối với khóa 2025.pdf
🎉🎉🎉 HOÀN TẤT: Đã tải thành công 16 file PDF.


In [14]:
# Xem nội dung phần tử đầu tiên
if documents:
    print("\n📄 Nội dung phần tử đầu tiên:")
    print(documents[0].page_content)

# In ra cấu trúc của document đầu tiên để hiểu rõ hơn
if documents:
    print("\n🔍 Cấu trúc phần tử đầu tiên:")
    print(documents[0])

# Xem toàn bộ nội dung của page_content các phần tử
for i, doc in enumerate(documents):
    print(f"\n--- Phần tử {i+1} ---")
    print(doc.page_content)


📄 Nội dung phần tử đầu tiên:
£({

BỘ GIÁO DỤC VÀ ĐÀO TẠO

TRƯỜNG ĐẠI HỌC SƯ PHẠM KỸ THUẬT THÀNH PHÔ HÒ CHÍ MINH

CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc

PEnBNnnononagvoouoe

Sô:4 /QD-ĐHSPKT

TP.Hồ Chí Minh, ngày {{ tháng 5 năm 2023

QUYÉT ĐỊNH

Về việc ban hành quy định các học phần ngoại ngữ trong chương trình đào tạo trình độ đại học từ khóa 2023

HIỆU TRƯỞNG TRƯỜNG ĐẠI HỌC SƯ PHẠM KỸ THUẬT TP. HÒ CHÍ MINH

Căn cứ Luật Giáo dục đại học ngày 18 tháng 6 năm 2012 và Luật sửa đổi, bổ Sung một số điều của Luật Giáo dục đại học ngày 19 tháng 11 năm 2018;

Căn cứ Nghị định 99/2019/NĐ-CP ngày 30 tháng 12 năm 2019 của Chính phủ Quy định chỉ tiết và hướng dẫn thi hành một số điều của Luật sửa đổi, bỏ sung một số điều của Luật giáo dục đại học;

Căn cứ Quyết định số 937/QĐ-TTg ngày 30 tháng 6 năm 2017 của Thủ tướng Chính phủ về việc phê duyệt đề án thí điểm đổi mới cơ chế hoạt động của Trường Đại học Sư phạm Kỹ thuật TP. Hô Chí Minh;

Căn cứ Nghị quyết số 11/NQ-HĐT ngày

In [ ]:
# # Truong - Dùng DocumentConverter từ docling (tốt nhất cho bảng + tiếng Việt)
# from docling.document_converter import DocumentConverter, PdfFormatOption
# from docling.datamodel.base_models import InputFormat
# from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode
# from langchain_core.documents import Document
# import os

# def load_pdf_docs_with_docling(data_dir: str, max_files: int = 2):
#     """Load PDF bằng Docling DocumentConverter (tốt cho bảng)"""
    
#     # Cấu hình pipeline cho PDF
#     pipeline_options = PdfPipelineOptions()
#     pipeline_options.do_ocr = True  # Bật OCR cho scan
#     pipeline_options.do_table_structure = True  # QUAN TRỌNG: Phát hiện cấu trúc bảng
#     pipeline_options.table_structure_options.do_cell_matching = True  # Match cells trong bảng
#     pipeline_options.table_structure_options.mode = TableFormerMode.ACCURATE  # Chế độ chính xác
    
#     # Cấu hình OCR tiếng Việt
#     try:
#         pipeline_options.ocr_options.lang = ["vie"]
#     except:
#         print("⚠️ Không thể cấu hình OCR tiếng Việt, sử dụng mặc định.")
    
#     # Khởi tạo converter
#     converter = DocumentConverter(
#         format_options={
#             InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
#         }
#     )
    
#     docs = []
#     count = 0
    
#     for filename in os.listdir(data_dir):
#         if not filename.lower().endswith(".pdf"):
#             continue
#         if max_files is not None and count >= max_files:
#             break
        
#         file_path = os.path.join(data_dir, filename)
#         print(f"📄 [{count+1}/{max_files}] Đang convert: {filename}")
        
#         try:
#             # Convert PDF
#             result = converter.convert(file_path)
            
#             # Export sang Markdown (giữ nguyên cấu trúc bảng)
#             markdown_text = result.document.export_to_markdown()
            
#             # Loại bỏ placeholder ảnh nếu có
#             lines = [line for line in markdown_text.split('\n') 
#                     if not line.strip().startswith('<!-- image')]
#             clean_text = '\n'.join(lines).strip()
            
#             if len(clean_text) < 50:
#                 print(f"  ⚠️ File quá ít nội dung")
#             else:
#                 docs.append(Document(
#                     page_content=clean_text,
#                     metadata={"source": file_path, "filename": filename}
#                 ))
#                 print(f"  ✅ Hoàn tất: {len(clean_text)} ký tự")
                
#                 # In preview để kiểm tra bảng
#                 preview = clean_text[:500].replace('\n', '\\n')
#                 print(f"  📝 Preview: {preview}...")
            
#             count += 1
            
#         except Exception as e:
#             print(f"  ❌ Lỗi: {e}")
#             import traceback
#             traceback.print_exc()
    
#     print(f"\n🎉 Tổng: {len(docs)} file được tải thành công.")
#     return docs

# # Chạy
# data_dir = "./data"
# documents = load_pdf_docs_with_docling(data_dir, max_files=2)

# print(f"\n🎉 HOÀN TẤT: Đã tải {len(documents)} file PDF.")

# # Kiểm tra kết quả
# if documents:
#     print("\n" + "="*80)
#     print("📋 KIỂM TRA KẾT QUẢ - Document đầu tiên:")
#     print("="*80)
#     print(documents[0].page_content[:1000])  # In 1000 ký tự đầu
# # ...existing code...

2025-11-13 21:30:29,522 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-13 21:30:29,530 - INFO - Going to convert document batch...
2025-11-13 21:30:29,531 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 21cbf929b214b9669a3f4383152eb367


📄 [1/2] Đang convert: 1238 qd ban hanh quy dinh cac hoc phan ngoai ngu trong chuong trinh dao tao trinh do DH tu khoa 2023.pdf


2025-11-13 21:30:30,280 - INFO - Accelerator device: 'cpu'
[INFO] 2025-11-13 21:30:30,301 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-11-13 21:30:30,322 [RapidOCR] download_file.py:60: File exists and is valid: D:\LearnUniversity\Semester11\TieuLuanChuyenNganh\LLM_University_Policy_Chatbot\venv\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-11-13 21:30:30,323 [RapidOCR] main.py:53: Using D:\LearnUniversity\Semester11\TieuLuanChuyenNganh\LLM_University_Policy_Chatbot\venv\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-11-13 21:30:30,422 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-11-13 21:30:30,435 [RapidOCR] download_file.py:60: File exists and is valid: D:\LearnUniversity\Semester11\TieuLuanChuyenNganh\LLM_University_Policy_Chatbot\venv\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-11-13 21:30:30,438 [RapidOCR] main.py:53: Using D:\LearnUniversity\Semeste

  ✅ Hoàn tất: 8549 ký tự
  📝 Preview: ## BQGIAODUCVADAOTAO\n\n## TRUONGDAIHQCSU'PHAMKY THUAT THANH PHOHO CHIMINH\n\nS6: IQ 38/QD-DHSPKT\n\n## CONGHOAXAHQI CHUNGHIAVIETNAM Doc lap -Ty do -Hanh phuc\n\nTP.Ho Chi Minh, ngay M1 thang 5 nam 2023\n\n## QUYET DINH\n\nVe viec ban hanh quy dinh cac hoc phan ngoai ngir trongchuo'ng trinh dao tao trinh dodaihocturkhoa 2023\n\n## HIEU TRUONG TRUONG DAI HQC SU' PHAM KY THUAT TP.HO CHI MINH\n\nCan cir Luat Giao duc dai hoc ngay 18 thang 6 nam 2012 va Luat sira doi, bo sung mot sodieu cua Luat Giaoduc dai hoc nga...
📄 [2/2] Đang convert: 16.9.2025_TB triển khai KS SV về HĐGD của GV_HKI 25-26 final.signed.pdf


2025-11-13 21:31:08,421 - INFO - Going to convert document batch...
2025-11-13 21:31:08,422 - INFO - Processing document 16.9.2025_TB triển khai KS SV về HĐGD của GV_HKI 25-26 final.signed.pdf
2025-11-13 21:31:11,957 - INFO - Finished converting document 16.9.2025_TB triển khai KS SV về HĐGD của GV_HKI 25-26 final.signed.pdf in 4.17 sec.


  ✅ Hoàn tất: 4663 ký tự
  📝 Preview: ## TRƯỜNG ĐẠI HỌC SƯ PHẠM KỸ THUẬT THÀNH PHỐ HỒ CHÍ MINH\n\n## PHÒNG KHẢO THÍ &amp; ĐẢM BẢO CHẤT LƯỢNG\n\nSố : 05/TB-KT&amp; ĐBCL\n\nTP. Hồ Chí Minh, ngày 15 tháng 09 năm 20 25\n\n## THÔNG BÁO\n\n## V/v triển khai khảo sát lấy ý kiến sinh viên về hoạt động giảng dạy của Giảng viên HKI năm học 2025-2026\n\n## Kính gửi: S\n\ninh viên hệ chính quy trong toàn trường\n\nNhằm nâng cao chất lượng giảng dạy và cải thiện trải nghiệm học tập, Nhà trường triển khai khảo sát lấy ý kiến sinh viên về hoạt động giảng dạy của ...

🎉 Tổng: 2 file được tải thành công.

🎉 HOÀN TẤT: Đã tải 2 file PDF.

📋 KIỂM TRA KẾT QUẢ - Document đầu tiên:
## BQGIAODUCVADAOTAO

## TRUONGDAIHQCSU'PHAMKY THUAT THANH PHOHO CHIMINH

S6: IQ 38/QD-DHSPKT

## CONGHOAXAHQI CHUNGHIAVIETNAM Doc lap -Ty do -Hanh phuc

TP.Ho Chi Minh, ngay M1 thang 5 nam 2023

## QUYET DINH

Ve viec ban hanh quy dinh cac hoc phan ngoai ngir trongchuo'ng trinh dao tao trinh dodaihocturkhoa 2023

## HIEU TRUONG T

## 4. Chia nhỏ tài liệu thành chunks

In [18]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Mỗi chunk tối đa 1000 ký tự
    chunk_overlap=200,    # Giữ lại 200 ký tự chồng lắp giữa các chunk
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]  # Thứ tự ưu tiên cắt
)

print("✅ Đã khởi tạo RecursiveCharacterTextSplitter (thông dụng).")

# # Thử cách 2 để phù hợp với markdown hiểu bảng hơn
# text_splitter = MarkdownTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=100  # Markdown overlap có thể nhỏ hơn
# )

# print("✅ Đã khởi tạo MarkdownTextSplitter (thông minh hơn cho bảng).")

chunks = text_splitter.split_documents(documents)
print(f"✅ Đã chia thành {len(chunks)} chunks!")
print(f"\n📊 Thống kê:")
print(f"   - Số lượng documents gốc: {len(documents)}")
print(f"   - Số lượng chunks sau khi chia: {len(chunks)}")

# Xem nội dung chunk đầu tiên
if chunks:
    print(f"\n📝 Nội dung chunk đầu tiên ({len(chunks[0].page_content)} ký tự):")
    print(chunks[0].page_content)

✅ Đã khởi tạo RecursiveCharacterTextSplitter (thông dụng).
✅ Đã chia thành 158 chunks!

📊 Thống kê:
   - Số lượng documents gốc: 16
   - Số lượng chunks sau khi chia: 158

📝 Nội dung chunk đầu tiên (933 ký tự):
£({

BỘ GIÁO DỤC VÀ ĐÀO TẠO

TRƯỜNG ĐẠI HỌC SƯ PHẠM KỸ THUẬT THÀNH PHÔ HÒ CHÍ MINH

CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc

PEnBNnnononagvoouoe

Sô:4 /QD-ĐHSPKT

TP.Hồ Chí Minh, ngày {{ tháng 5 năm 2023

QUYÉT ĐỊNH

Về việc ban hành quy định các học phần ngoại ngữ trong chương trình đào tạo trình độ đại học từ khóa 2023

HIỆU TRƯỞNG TRƯỜNG ĐẠI HỌC SƯ PHẠM KỸ THUẬT TP. HÒ CHÍ MINH

Căn cứ Luật Giáo dục đại học ngày 18 tháng 6 năm 2012 và Luật sửa đổi, bổ Sung một số điều của Luật Giáo dục đại học ngày 19 tháng 11 năm 2018;

Căn cứ Nghị định 99/2019/NĐ-CP ngày 30 tháng 12 năm 2019 của Chính phủ Quy định chỉ tiết và hướng dẫn thi hành một số điều của Luật sửa đổi, bỏ sung một số điều của Luật giáo dục đại học;

Căn cứ Quyết định số 937/QĐ-TTg ngày 30 tháng 6

In [ ]:
# Sau khi chunking vẫn có các câu trong 1 chunk bị xuống dòng 2 lần, cần xử lý thêm để loại bỏ các xuống dòng thừa này
def clean_text(text):
    # Bước 1: Thay thế \n\n thành \n
    text = re.sub(r'\n\n', '\n', text)
    
    # Bước 2: Sửa lỗi OCR cụ thể - tạm thời
    text = text.replace("PHÔ HÒ ", "PHỐ HỒ")
    
    # Bước 3: QUAN TRỌNG - Chỉ xóa khoảng trắng thừa, GIỮ LẠI \n
    # Thay vì dùng \s+ (match tất cả whitespace bao gồm \n)
    # Chỉ thay thế nhiều khoảng trắng/tab liên tiếp, KHÔNG thay \n
    text = re.sub(r'[ \t]+', ' ', text)  # Chỉ match khoảng trắng ' ' và tab '\t', KHÔNG match \n
    
    # Bước 4: Xóa khoảng trắng ở đầu và cuối mỗi dòng (nhưng giữ \n)
    lines = text.split('\n')
    lines = [line.strip() for line in lines]
    text = '\n'.join(lines)
    
    return text

# Áp dụng cho chunks
for chunk in chunks:
    chunk.page_content = clean_text(chunk.page_content)

print("✅ Tiền xử lý tạm thời hoàn tất cho tất cả chunks.")

✅ Tiền xử lý tạm thời hoàn tất cho tất cả chunks.


In [20]:
# Nội dung của toàn bộ chunk
for i, chunk in enumerate(chunks):
    print(f"\n--- Chunk {i+1} ({len(chunk.page_content)} ký tự) ---")
    print(chunk.page_content)
    print(chunk.metadata)


--- Chunk 1 (920 ký tự) ---
£({
BỘ GIÁO DỤC VÀ ĐÀO TẠO
TRƯỜNG ĐẠI HỌC SƯ PHẠM KỸ THUẬT THÀNH PHỐ HỒCHÍ MINH
CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc
PEnBNnnononagvoouoe
Sô:4 /QD-ĐHSPKT
TP.Hồ Chí Minh, ngày {{ tháng 5 năm 2023
QUYÉT ĐỊNH
Về việc ban hành quy định các học phần ngoại ngữ trong chương trình đào tạo trình độ đại học từ khóa 2023
HIỆU TRƯỞNG TRƯỜNG ĐẠI HỌC SƯ PHẠM KỸ THUẬT TP. HÒ CHÍ MINH
Căn cứ Luật Giáo dục đại học ngày 18 tháng 6 năm 2012 và Luật sửa đổi, bổ Sung một số điều của Luật Giáo dục đại học ngày 19 tháng 11 năm 2018;
Căn cứ Nghị định 99/2019/NĐ-CP ngày 30 tháng 12 năm 2019 của Chính phủ Quy định chỉ tiết và hướng dẫn thi hành một số điều của Luật sửa đổi, bỏ sung một số điều của Luật giáo dục đại học;
Căn cứ Quyết định số 937/QĐ-TTg ngày 30 tháng 6 năm 2017 của Thủ tướng Chính phủ về việc phê duyệt đề án thí điểm đổi mới cơ chế hoạt động của Trường Đại học Sư phạm Kỹ thuật TP. Hô Chí Minh;
{'source': './data\\1238 qd ban hanh quy dinh cac 

## 5. Khởi tạo mô hình Embedding đa ngôn ngữ

In [21]:
print("⏳ Đang tải mô hình embedding... (Lần đầu sẽ tải về ~500MB)")

# Sử dụng mô hình đa ngôn ngữ, hỗ trợ tốt tiếng Việt
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'},  # Sử dụng CPU (thay 'cuda' nếu có GPU)
    encode_kwargs={'normalize_embeddings': True}  # Chuẩn hóa vectors
)

print("✅ Mô hình embedding đã sẵn sàng!")
print(f"📌 Tên mô hình: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
print(f"🌍 Hỗ trợ: Tiếng Việt và 50+ ngôn ngữ khác")
print(f"💾 Chạy hoàn toàn offline sau khi tải về")

2025-11-14 08:46:56,464 - INFO - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


⏳ Đang tải mô hình embedding... (Lần đầu sẽ tải về ~500MB)
✅ Mô hình embedding đã sẵn sàng!
📌 Tên mô hình: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
🌍 Hỗ trợ: Tiếng Việt và 50+ ngôn ngữ khác
💾 Chạy hoàn toàn offline sau khi tải về


In [22]:
# LỌC CÁC CHUNK RÁC
print(f"🔍 Bắt đầu lọc chunk. Số chunk ban đầu: {len(chunks)}")

# Đặt độ dài tối thiểu. 50 ký tự là một lựa chọn an toàn 
# (khoảng 10 từ tiếng Việt), đủ để loại bỏ rác 
# như '_', '---', 'Nơi nhận:', 'Trân trọng.' 
# mà không ảnh hưởng đến nội dung thực.
min_chunk_length = 50 

original_chunk_count = len(chunks)

# Lọc và giữ lại các chunk có độ dài LỚN HƠN hoặc BẰNG min_length
filtered_chunks = [chunk for chunk in chunks if len(chunk.page_content.strip()) >= min_chunk_length]

print(f"✅ Lọc hoàn tất.")
print(f"   - Đã loại bỏ: {original_chunk_count - len(filtered_chunks)} chunk rác (ngắn hơn {min_chunk_length} ký tự).")
print(f"   - Số chunk sạch còn lại: {len(filtered_chunks)}")

🔍 Bắt đầu lọc chunk. Số chunk ban đầu: 158
✅ Lọc hoàn tất.
   - Đã loại bỏ: 2 chunk rác (ngắn hơn 50 ký tự).
   - Số chunk sạch còn lại: 156


## 6. Tạo ChromaDB vector store và lưu trữ documents

In [23]:
# Phải làm phẳng (flatten) metadata thì mới lưu vào ChromaDB được
# print("🔧 Đang 'làm phẳng' (flatten) metadata để tương thích với ChromaDB...")

# # Lặp qua từng chunk (là một đối tượng Document)
# for chunk in filtered_chunks:
#     # Lấy metadata của chunk đó
#     metadata = chunk.metadata
    
#     # Lặp qua từng cặp key-value trong metadata
#     for key, value in metadata.items():
#         # Nếu giá trị (value) KHÔNG PHẢI là kiểu dữ liệu đơn giản
#         if not isinstance(value, (str, int, float, bool, type(None))):
#             # Hãy biến nó thành một chuỗi (string)
#             metadata[key] = str(value)
# print("✅ Metadata đã được làm phẳng.")

import gc # Garbage collector để giải phóng bộ nhớ
import time
# Xóa database cũ nếu tồn tại (để demo sạch)

# Đóng vectorstore cũ nếu có
if 'vectorstore' in globals():
    print("🔒 Đang đóng vectorstore cũ...")
    try:
        del vectorstore
        gc.collect()
        time.sleep(1)
        print("✅ Đã đóng vectorstore cũ")
    except:
        pass

db_path = "./chroma_db"
collection_name = "academic_regulations"

print(f"📦 Sử dụng collection: {collection_name}")

# KIỂM TRA: Nếu collection đã tồn tại, xóa nó trước
if os.path.exists(db_path):
    try:
        import chromadb
        client = chromadb.PersistentClient(path=db_path)
        
        # Lấy danh sách collections hiện có
        existing_collections = [col.name for col in client.list_collections()]
        
        if collection_name in existing_collections:
            print(f"🗑️ Collection '{collection_name}' đã tồn tại, đang xóa...")
            try:
                client.delete_collection(name=collection_name)
                print(f"✅ Đã xóa collection '{collection_name}' cũ")
                time.sleep(0.5)  # Đợi một chút
            except Exception as e:
                print(f"⚠️ Không thể xóa collection: {e}")
                print(f"💡 Sẽ tạo collection với tên mới: {collection_name}_new")
                collection_name = f"{collection_name}_new"
        else:
            print(f"ℹ️ Collection '{collection_name}' chưa tồn tại, sẽ tạo mới")
        
        # Đóng client
        del client
        gc.collect()
        time.sleep(0.5)
        
    except ImportError:
        print("⚠️ Không thể import chromadb, sẽ bỏ qua bước kiểm tra collection")
    except Exception as e:
        print(f"⚠️ Lỗi khi kiểm tra collection: {e}")

print("⏳ Đang tạo embeddings và lưu vào ChromaDB... (Có thể mất vài phút)")

# Tạo vector store với persistent storage
vectorstore = Chroma.from_documents( 
    documents=filtered_chunks,
    embedding=embedding_model,
    persist_directory=db_path, # Lưu database vào thư mục local, giữ lại sau khi tắt notebook
    collection_name=collection_name # Tên collection trong ChromaDB để quản lý nhiều bộ documents khác nhau
)

print(f"✅ Đã lưu {len(filtered_chunks)} filtered_chunks vào ChromaDB!")
print(f"💾 Database được lưu tại: {os.path.abspath(db_path)}")
print(f"📦 Collection name: {collection_name}")

📦 Sử dụng collection: academic_regulations


2025-11-14 08:47:09,653 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


🗑️ Collection 'academic_regulations' đã tồn tại, đang xóa...
✅ Đã xóa collection 'academic_regulations' cũ
⏳ Đang tạo embeddings và lưu vào ChromaDB... (Có thể mất vài phút)
✅ Đã lưu 156 filtered_chunks vào ChromaDB!
💾 Database được lưu tại: d:\LearnUniversity\Semester11\TieuLuanChuyenNganh\LLM_University_Policy_Chatbot\chroma_db
📦 Collection name: academic_regulations


## 7. Kiểm tra hệ thống RAG với truy vấn tiếng Việt

In [24]:
query = "cho tôi biết về thời gian đkmh?"

print(f"🔍 Truy vấn: '{query}'")
print("⏳ Đang tìm kiếm các chunks liên quan nhất...\n")

# Tìm kiếm 1 documents tương đồng nhất
results = vectorstore.similarity_search(query, k=3)

print(f"✅ Tìm thấy {len(results)} kết quả liên quan:\n")
print("=" * 80)

for i, doc in enumerate(results, 1):
    print(f"\n📄 KẾT QUẢ #{i}")
    print(f"📝 Nội dung của chunk liên quan:\n{doc.page_content}")
    print(f"📌 Metadata: {doc.metadata}")
    print("=" * 80)

🔍 Truy vấn: 'cho tôi biết về thời gian đkmh?'
⏳ Đang tìm kiếm các chunks liên quan nhất...

✅ Tìm thấy 3 kết quả liên quan:


📄 KẾT QUẢ #1
📝 Nội dung của chunk liên quan:
2. Các mốc thời gian đăng ký môn học thay thế trong học kỷ:
-_ Đợt 1: Nhận trong 02 tuân dầu học kỳ.
Cụ thể: từ ngày 11/08/2025 dến hết ngày 23/08/2025.
-_ Đợt 2: Nhận trong 01 tuần của học kỳ.
Cụ thể: từ ngày 27/10/2025 dến hết ngày (01/11/2025.
3. Sinh viên thực hiện các bước đăng ký theo “Phụ lục” dính kèm.
4. Sau khi đăng ký trên trang online, sinh viên theo dõi kết quả duyệt dơn trên n trang Online mục “Thông báo cá nhân”; phần “Đăng ký các mẫu đơn — Loại đăng ký — Thay thể
II. ĐÓI VỚI KHOA/BỘ MÔN, PHÒNG ĐÀO TẠO
¬ -_ Các Khoa/Bộ môn quản môn học, Khoa Đào tạo Quôc tê duyệt đơn xin thay thế môn họ Ó
của các sinh viên thuộc Khoa/Bộ môn.
-_ Thời gian: đợt 1 từ ngày 25-31/08/2025 và đợt 2 từ ngày 04-10/11/2025.
-_ Phòng Đào tạo duyệt đơn sau khi Khoa duyệt.
-_ Dự kiến công bố kết quả duyệt đơn xin thay thế môn học củ